# Préparation

In [98]:
%cd /content
!rm -rf test-zoomLDM
!git clone https://github.com/misterstab/test-zoomLDM.git
%cd test-zoomLDM

/content
Cloning into 'test-zoomLDM'...
remote: Enumerating objects: 214, done.
remote: Counting objects: 100% (214/214), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 214 (delta 99), reused 165 (delta 60), pack-reused 0 (from 0)
Receiving objects: 100% (214/214), 23.09 MiB | 22.30 MiB/s, done.
Resolving deltas: 100% (99/99), done.
/content/test-zoomLDM


In [100]:
import sys
sys.path.append('/content/test-zoomLDM')
sys.path.insert(0, "../")

# Import des librairies nécessaires

In [101]:
!pip install -r requirements.txt

Obtaining clip from git+https://github.com/openai/CLIP.git@main#egg=clip (from -r requirements.txt (line 49))
  Cloning https://github.com/openai/CLIP.git (to revision main) to ./src/clip
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /content/test-zoomLDM/src/clip
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
Obtaining file:///content/test-zoomLDM (from -r requirements.txt (line 50))
  Preparing metadata (setup.py) ... done
  Attempting uninstall: stable-diffusion
    Found existing installation: stable-diffusion 0.0.1
    Uninstalling stable-diffusion-0.0.1:
      Successfully uninstalled stable-diffusion-0.0.1
  Running setup.py develop for stable-diffusion
  Running setup.py develop for clip


In [113]:
!pip install triton

In [114]:
from datasets import load_dataset
from omegaconf import OmegaConf

import numpy as np
import torch
import triton
from einops import rearrange
from huggingface_hub import hf_hub_download
from huggingface_hub import snapshot_download
from IPython.utils import io
from torch.utils.data import DataLoader

from ldm.util import instantiate_from_config
from utils import collate_fn

from PIL import Image

from large_image_gen.resizer import Resizer
from large_image_gen.utils import model_pred, decode_large_image

import torch.nn.functional as F
from tqdm.auto import tqdm

import matplotlib.pyplot as plt

In [115]:
model_dir = snapshot_download(repo_id="StonyBrook-CVLab/ZoomLDM")
print(f"Modèle téléchargé dans : {model_dir}")

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Modèle téléchargé dans : /root/.cache/huggingface/hub/models--StonyBrook-CVLab--ZoomLDM/snapshots/433088972f0e96398ff9c54967a21fc3303eb6a5


In [123]:
MAG = "1_25x" # The sample dataset has 20x, 10x, 5x, 2_5x and 1_25x data

ds = load_dataset("StonyBrook-CVLab/ZoomLDM-demo-dataset", data_dir="data", name=MAG, trust_remote_code=True, split='train')
dl = DataLoader(ds, batch_size=4, collate_fn = collate_fn)

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'StonyBrook-CVLab/ZoomLDM-demo-dataset' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'StonyBrook-CVLab/ZoomLDM-demo-dataset' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Resolving data files:   0%|          | 0/80 [00:00<?, ?it/s]

ValueError: BuilderConfig '1_25x' not found. Available: ['default']